# Lab 2

In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [2]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!ls /users/trush/CSC496/PageRank/Lab2

data  Lab2.ipynb


In [4]:
auth_log = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data/auth.log")
auth_log.take(50)

['Dec 13 00:06:16 submitty sshd[19509]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.184  user=root',
 'Dec 13 00:06:17 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:06:21 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:07:01 submitty CRON[19523]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Dec 13 00:07:02 submitty CRON[19523]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Dec 13 00:08:01 submitty CRON[19536]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Dec 13 00:08:01 submitty CRON[19536]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Dec 13 00:08:29 submitty sshd[19552]: Invalid user shalini from 27.128.173.81 port 49454',
 'Dec 13 00:08:29 submitty sshd[19552]: pam_unix(sshd:auth): check pass; user unknown',
 'Dec 13 00:08:29 s

In [5]:
auth_log1 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data/auth.log.1")
#auth_log1.take(5)

In [6]:
auth_log2 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data/auth.log.2")
#auth_log2.take(5)

In [7]:
auth_log3 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data/auth.log.3")
#auth_log3.take(5)

In [8]:
auth_log4 = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data/auth.log.4")
#auth_log4.take(5)

## 1. How many failed attempts to access the server as root are there? List all countries from which these attempts where carried out.

In [9]:
# Total failed attempts to login
#total_failed_attempts = auth_log.filter(lambda x: "sshd" in x)
#total_failed_attempts.count()

# Number of failed attempts to login as root
failed_attempts = auth_log.filter(lambda x: "root" in x and "sshd" in x)
failed_attempts.take(10)
# find distinct attempts by unique ip
#failed_attempts.count()

['Dec 13 00:06:16 submitty sshd[19509]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.184  user=root',
 'Dec 13 00:06:17 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:06:21 submitty sshd[19509]: Failed password for root from 218.92.0.184 port 3753 ssh2',
 'Dec 13 00:10:35 submitty sshd[19716]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.247  user=root',
 'Dec 13 00:10:36 submitty sshd[19716]: Failed password for root from 218.92.0.247 port 16026 ssh2',
 'Dec 13 00:10:40 submitty sshd[19716]: Failed password for root from 218.92.0.247 port 16026 ssh2',
 'Dec 13 00:12:41 submitty sshd[19833]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.93.207.88  user=root',
 'Dec 13 00:12:43 submitty sshd[19833]: Failed password for root from 218.93.207.88 port 46251 ssh2',
 'Dec 13 00:12:45 submitty sshd[1

In [10]:
import pycountry

def get_country_name(x):
    name = pycountry.countries.get(alpha_2=x)
    return name.name

In [11]:
# Countries where the failed attempts were carried out
# Split each literal to isolate the ip
# use python library to match ip to country
import re

countries_ip = failed_attempts.map(lambda line: re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=line)[1])
countries_ip.take(5)

#countries_names = countries_ip.map(lambda line: get_country_name(line))
#countries_names.take(10)

['218.92.0.184',
 '218.92.0.184',
 '218.92.0.184',
 '218.92.0.247',
 '218.92.0.247']

## 2. How many failed attempts to access the server as non-root users are there? What are the attempted usernames? List all countries from which these attempts where carried out. 

In [46]:
# Number of failed attempts to login as non-root
failed_attempts_nonroot = auth_log.filter(lambda x: "root" not in x and "sshd" in x and "user unknown" not in x)
failed_attempts_nonroot_ip = failed_attempts_nonroot.map(lambda line: re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=line)[1])
#failed_attempts.take(5)
#failed_attempts_nonroot.count()
#failed_attempts_nonroot_ip.take(5)
#failed_attempts_nonroot_usernames = failed_attempts_nonroot.map(lambda line: re.split(pattern=r'(user )', string=line).rsplit(' from'))
#failed_attempts_nonroot_usernames.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 34, pcvm603-3.emulab.net, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/rdd.py", line 1440, in takeUpToNumLeft
    yield next(iterator)
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py",

## 3. Through out the durations of the log files, which date has the highest number of attack attempts? Anything interesting about that particular date?

In [ ]:
# Something to do with Chinese Labor Day (May 1)